In [1]:
%run /home/bb/src/python/valkyrie/apps/common/run_notebook_import.py

/media/veracrypt1/python/valkyrie/apps/common/run_notebook_import.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import copy
from itertools import product as cartesian_product

from overrides import overrides
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.nn import functional as F

from valkyrie.tools import *
from valkyrie.ml import modules
from valkyrie.ml.utils import tensor, HyperParameters
from valkyrie.ml import utils as ml_utils
from valkyrie.nibelungen.data import DataMgr, Df2T2

/media/veracrypt1/python/valkyrie/lib/valkyrie/nibelungen/data.py:192: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [3]:
sdate, edate = '2023-01-01', '2023-01-31'
instr = 'BDM.BTC.USDT.FP'
freq = '1s'
ret_n_s = [30]

#########################################################
#DataMgr
#########################################################
data_mgr = DataMgr(sdate, edate, freq, '/home/bb/data/BDM', instrs = ['BDM.BTC.USDT.FP'], ret_n_s = [120])
df_res = data_mgr.get(instr) #'bpx_last','apx_last','bq_last','aq_last',
channels = ['buy_qty_sum', 'sell_qty_sum']
n_channels = len(channels)
df2t2 = Df2T2(df_res, M = 16, xcols = ['buy_qty_sum', 'sell_qty_sum'], ycol = 'mid_last_ret_120_n',
              wcol = 'wgt_mid_last_ret_120_n', mul = 2, dtype = torch.float32, device='cpu')

after dropping na 2678245 -> 2645478


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2645478 and the array at index 1 has size 2678245

In [ ]:
print_df_mem_usage(df_res)

In [ ]:
len(df2t2)

In [ ]:
from valkyrie.ml.data import DataModule

In [ ]:
dm = DataModule.from_dataset(df2t2, None, 2048)

In [ ]:
class LrNet(modules.Regressor): 
    def __init__(self, lr):
        print("Lr Net")
        super().__init__('l1')

        self.save_hyperparameters()
        self.net = nn.Sequential(
          nn.Flatten(),
          nn.LazyLinear(1, bias=False)
          )

In [ ]:
lr_net = LrNet(1e-3)

In [ ]:
net = LrNet(lr = 1e-2)
X = df2t2[0][0]
ml_utils.layer_summary(net, X.shape)
ml_utils.parameter_summary(net)

In [ ]:
trainer = modules.Trainer(max_epochs=2, num_gpus=1)
with Timer('training'):
    trainer.fit(net, dm)

In [ ]:
def eval_init_loss(net, dm):
    init_loss, loss = 0.0, 0.0
    dl = dm.get_dataloader(True)
    for i, batch in enumerate(dl):
        X, YW = batch[0], batch[-1]                
        ZEROS = torch.zeros(YW.shape[0])
        with torch.no_grad():            
            init_loss += net.loss(ZEROS, YW)    
            Y_hat = net(X.to(ml_utils.gpu()))
            loss += net.loss(Y_hat, YW.to(ml_utils.gpu()))
    return init_loss, loss

In [ ]:
eval_init_loss(net, dm)

In [ ]:
ml_utils.parameter_summary(net)

In [ ]:
for name, param in net.named_parameters():
    print(f"Parameter name: {name}, param: {param}")

In [ ]:
class Inception(nn.Module):
    # c1--c4 are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)
        

    def forward(self, x):        
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

class GoogleNet(modules.Regressor):
    def b1(self):
        return nn.Sequential(            
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
@d2l.add_to_class(GoogleNet)
def b2(self):
    return nn.Sequential(        
        nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(192, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b3(self):
    return nn.Sequential(        
        Inception(64, (96, 128), (16, 32), 32),
        Inception(128, (128, 192), (32, 96), 64),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(
                         Inception(192, (96, 208), (16, 48), 64),
                         Inception(160, (112, 224), (24, 64), 64),
                         Inception(128, (128, 256), (24, 64), 64),
                         Inception(112, (144, 288), (32, 64), 64),
                         Inception(256, (160, 320), (32, 128), 128),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(
                         Inception(192, (96, 208), (16, 48), 64),
                         Inception(160, (112, 224), (24, 64), 64),
                         Inception(128, (128, 256), (24, 64), 64),
                         Inception(112, (144, 288), (32, 64), 64),
                         Inception(256, (160, 320), (32, 128), 128),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b5(self):
    return nn.Sequential(
                         Inception(256, (160, 320), (32, 128), 128),
                         Inception(384, (192, 384), (48, 128), 128),
                         nn.AdaptiveAvgPool2d((1,1)), nn.Flatten())

@d2l.add_to_class(GoogleNet)
def b0(self):
    return nn.BatchNorm2d(2);



@d2l.add_to_class(GoogleNet)
def __init__(self, lr=0.1, num_classes=10):
    #super(GoogleNet, self).__init__()
    
    super(GoogleNet, self).__init__('l1')        
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b0(), self.b1(), self.b2(), self.b3(), self.b4(),
                             self.b5(), nn.LazyLinear(1))
    self.net.apply(d2l.init_cnn)

In [ ]:
class Residual(nn.Module):  #@save
    """The Residual block of ResNet models."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1,
                                   stride=strides)
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
    
class ResNet(modules.Regressor):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
@d2l.add_to_class(ResNet)
def block(self, num_residuals, num_channels, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels))
    return nn.Sequential(*blk)

@d2l.add_to_class(ResNet)
def __init__(self, arch, lr=0.1, num_classes=10):
    super(ResNet, self).__init__('l1')
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1())
    for i, b in enumerate(arch):
        self.net.add_module(f'b{i+2}', self.block(*b, first_block=(i==0)))
    self.net.add_module('last', nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)
    
class ResNet18(ResNet):
    def __init__(self, lr=0.1):
        super().__init__(((2, 64), (2, 128), (2, 256), (2, 512)),
                       lr, 1)

In [ ]:
res_net = ResNet18(lr = 1e-4)
X = df2t2[0][0]
ml_utils.layer_summary(res_net, X.shape)
ml_utils.parameter_summary(res_net)

In [ ]:
google_net = GoogleNet(lr = 0.1)
X = df2t2[0][0]
ml_utils.layer_summary(google_net, X.shape)
ml_utils.parameter_summary(google_net)

In [ ]:
trainer = modules.Trainer(max_epochs=100, num_gpus=1)
with Timer('training'):
    trainer.fit(google_net, dm)

In [ ]:
def eval_init_loss(net, dm):
    init_loss, loss = 0.0, 0.0
    dl = dm.get_dataloader(True)
    for i, batch in enumerate(dl):
        X, YW = batch[0], batch[-1]
        ZEROS = torch.zeros(YW.shape[0])
        with torch.no_grad():
            init_loss += net.loss(ZEROS, YW)
            Y_hat = net(X.to(ml_utils.gpu()))
            loss += net.loss(Y_hat, YW.to(ml_utils.gpu()))
            #Y_hat = lr.predict(X.view(-1, 6)) #net(X.to(ml_utils.gpu())).to(ml_utils.cpu())
            #Y_hat = torch.tensor(Y_hat)
            #Y_hat = YW[:,0]
            #loss += net.loss(Y_hat, YW)                        
    return init_loss, loss

In [ ]:
eval_init_loss(google_net, dm)

In [ ]:
trainer = modules.Trainer(max_epochs=100, num_gpus=1)
with Timer('training'):
    trainer.fit(google_net, dm)

In [ ]:
dl = dm.get_dataloader(train=True)

In [ ]:
dl.dataset[10]

In [ ]:
random_batch = next(iter(dl))

In [ ]:
random_batch[0].shape

In [ ]:
random_batch[1].shape

In [ ]:
x.shape

In [ ]:
from valkyrie.ml.utils import to_gpu

In [ ]:
model = google_net
dl = dm.get_dataloader(train=True)

# Define loss function and optimizer (assuming Mean Squared Error and SGD optimizer)
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop (Example: training for 100 epochs)
num_epochs = 1000
for epoch in range(num_epochs):
    print(f'epoch = {epoch}')
    # Forward pass: Compute predicted y by passing x to the model
    iterator = iter(dl)
    epoch_loss = 0.0
    for batch_idx, random_batch in enumerate(dl):           
        x = random_batch[0]
        y = random_batch[1][:,0]
        x = to_gpu(x)
        y = to_gpu(y)
        outputs = model(x)
        # Compute the loss
        loss = criterion(outputs.squeeze(), y)  # Assuming single-value prediction
        epoch_loss += loss        
        # Zero gradients, backward pass, and optimize
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()
        
    print(epoch_loss)